In [2]:
import os
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import kagglehub

In [3]:
path=kagglehub.dataset_download("zalando-research/fashionmnist")

100%|██████████| 68.8M/68.8M [00:00<00:00, 179MB/s]

Extracting files...


In [4]:
os.listdir(path)

['fashion-mnist_test.csv',
 'train-images-idx3-ubyte',
 't10k-labels-idx1-ubyte',
 'train-labels-idx1-ubyte',
 't10k-images-idx3-ubyte',
 'fashion-mnist_train.csv']

In [6]:
train_df=pd.read_csv(path+"/fashion-mnist_train.csv")
test_df=pd.read_csv(path+"/fashion-mnist_test.csv")

In [8]:
X=train_df.drop("label",axis=1)
y=train_df['label']
X_=test_df.drop("label",axis=1)
y_=test_df["label"]

In [9]:
X=X/255.0
X_=X_/255.0

In [10]:
x_train_torch=torch.tensor(X.values,dtype=torch.float32)
x_test_torch=torch.tensor(X_.values,dtype=torch.float32)
y_train_torch=torch.tensor(y.values,dtype=torch.long)
y_test_torch=torch.tensor(y_.values,dtype=torch.long)

In [11]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [13]:
x_train=x_train_torch.to(device)
y_train=y_train_torch.to(device)
x_test=x_test_torch.to(device)
y_test=y_test_torch.to(device)

In [44]:
class CustomDataset(Dataset):
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index],self.y[index]
x_train_dataset=CustomDataset(x_train,y_train)
y_test_dataset=CustomDataset(x_test,y_test)

In [45]:
class MYNN(nn.Module):
    def __init__(self,input_dim,output_dim,dropout,num_hidden,num_neuron):
        super().__init__()
        layers=[]
        for i in range(num_hidden):
            layers.append(nn.Linear(input_dim,num_neuron))
            layers.append(nn.BatchNorm1d(num_neuron))
            layers.append(nn.Dropout(dropout))
            layers.append(nn.ReLU())
            input_dim=num_neuron
        layers.append(nn.Linear(input_dim,output_dim))
        self.model=nn.Sequential(*layers)
    def forward(self,x):
        return self.model(x)

In [46]:
import torch.optim as optim

In [47]:
def objective(trial):
    num_hidden=trial.suggest_int("num_hidden_layers",1,6)
    num_neuron=trial.suggest_int("num_neuron",8,128,step=8)
    epochs=trial.suggest_int("epochs",10,50,step=5)
    learning_rate=trial.suggest_float("learning_rate",1e-4,1e-1,log=True)
    dropout=trial.suggest_float("dropout",0.1,0.5,step=0.1)
    batch_size=trial.suggest_int("batch_size",32,256,step=32)
    optimizer_name=trial.suggest_categorical("optimizer",["Adam","RMSprop","SGD"])
    weight_decay=trial.suggest_float("weight_decay",1e-4,1e-1,log=True)

    input_dim=784
    output_dim=10

    train_load=DataLoader(x_train_dataset,batch_size=batch_size,shuffle=True)
    test_load=DataLoader(y_test_dataset,batch_size=batch_size,shuffle=False)

    model=MYNN(input_dim,output_dim,dropout,num_hidden,num_neuron)
    model.to(device)

    criterion=nn.CrossEntropyLoss()

    if optimizer_name=="Adam":
        optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay,lr=learning_rate)
    elif optimizer_name=="SGD":
        optimizer=optim.SGD(model.parameters(),weight_decay=weight_decay,lr=learning_rate)
    else:
        optimizer=optim.RMSprop(model.parameters(),weight_decay=weight_decay,lr=learning_rate)
    for epoch in range(epochs):
        for x,y in train_load:
            x,y=x.to(device),y.to(device)
            out=model(x)
            loss=criterion(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    total=0
    correct=0
    with torch.no_grad():
        for x_tes,y_tes in test_load:
            x_tes,y_tes=x_tes.to(device),y_tes.to(device)
            outputs=model(x_tes)
            _,predicted=torch.max(outputs,1)
            total+=y_tes.size(0)
            correct+=(predicted==y_tes).sum().item()
        accuracy=correct/total
    return accuracy

In [48]:
import optuna

In [49]:
study=optuna.create_study(direction="maximize")
study.optimize(objective,n_trials=50)

[I 2025-12-29 17:17:48,359] A new study created in memory with name: no-name-5ae315ed-27d3-43ca-ba66-64451fde5408
[I 2025-12-29 17:18:28,474] Trial 0 finished with value: 0.7951 and parameters: {'num_hidden_layers': 1, 'num_neuron': 64, 'epochs': 45, 'learning_rate': 0.010025564996630303, 'dropout': 0.5, 'batch_size': 192, 'optimizer': 'Adam', 'weight_decay': 0.00640704984969735}. Best is trial 0 with value: 0.7951.
[I 2025-12-29 17:20:03,888] Trial 1 finished with value: 0.7521 and parameters: {'num_hidden_layers': 1, 'num_neuron': 48, 'epochs': 30, 'learning_rate': 0.004213344212114724, 'dropout': 0.5, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.02805901748663848}. Best is trial 0 with value: 0.7951.
[I 2025-12-29 17:20:24,011] Trial 2 finished with value: 0.8638 and parameters: {'num_hidden_layers': 3, 'num_neuron': 40, 'epochs': 20, 'learning_rate': 0.0007935587500282578, 'dropout': 0.4, 'batch_size': 224, 'optimizer': 'Adam', 'weight_decay': 0.00012606510396789887}

In [50]:
study.best_value

0.8973

In [51]:
study.best_params

{'num_hidden_layers': 6,
 'num_neuron': 104,
 'epochs': 45,
 'learning_rate': 0.00021326420310533226,
 'dropout': 0.2,
 'batch_size': 160,
 'optimizer': 'Adam',
 'weight_decay': 0.0007143925115484014}